
### Casper's Ghost Kitchen Initializer

Select `Run All` to create the job `Casper's Initializer`. Once this notebook has been run you can initialize the Casper's environment using the job, which also allows you to selectively create only parts of the demo (broken into tasks within the job)

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
dbutils.library.restartPython()

In [0]:
CATALOG = dbutils.widgets.get("CATALOG")

In [0]:
from databricks.sdk.service.jobs import JobSettings as Job
import os

Casper_s_Initializer = Job.from_dict(
    {
        "name": "Casper's Initializer",
        "tasks": [
            {
                "task_key": "Raw_Data",
                "notebook_task": {
                    "notebook_path": os.path.abspath("./notebooks/raw_data"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Lakeflow",
                "depends_on": [
                    {
                        "task_key": "Raw_Data",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./notebooks/lakeflow"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Refund_Recommender_Agent",
                "depends_on": [
                    {
                        "task_key": "Lakeflow",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./notebooks/refunder_agent"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Refund_Recommender_Stream",
                "depends_on": [
                    {
                        "task_key": "Refund_Recommender_Agent",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./notebooks/refunder_stream"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Lakebase",
                "depends_on": [
                    {
                        "task_key": "Refund_Recommender_Stream",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./notebooks/lakebase"),
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "App_Refund_Manager",
                "depends_on": [
                    {
                        "task_key": "Lakebase",
                    },
                ],
                "notebook_task": {
                    "notebook_path": os.path.abspath("./notebooks/apps"),
                    "source": "WORKSPACE",
                },
            },
        ],
        "queue": {
            "enabled": True,
        },
        "parameters": [
            {
                "name": "CATALOG",
                "default": f"{CATALOG}",
            },
            {
                "name": "EVENTS_VOLUME",
                "default": "events",
            },
            {
                "name": "LLM_MODEL",
                "default": "databricks-meta-llama-3-3-70b-instruct",
            },
            {
                "name": "REFUND_AGENT_ENDPOINT_NAME",
                "default": "caspers_refund_agent",
            },
            {
                "name": "SIMULATOR_SCHEMA",
                "default": "simulator",
            },
        ],
        "performance_target": "PERFORMANCE_OPTIMIZED",
    }
)

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
w.jobs.create(**Casper_s_Initializer.as_shallow_dict())